# Introduction to Q# #

Q# (Q-sharp) is a domain-specific programming language, part of [Microsoft's Quantum Development Kit (QDK)](https://docs.microsoft.com/en-us/quantum/), used for expressing quantum algorithms. It is to be used for writing subroutines that execute on an adjunct quantum processing unit (QPU), under the control of a classical host program and computer.

Operations and functions are the basic unit of execution in Q#. They are roughly equivalent to a function in C or C++ or Python, or a static method in C# or Java.

A Q# operation is a quantum subroutine. That is, it is a callable routine that contains quantum operations.

A Q# function is a classical subroutine used within a quantum algorithm. It may contain classical code but no quantum operations. Specifically, functions may not allocate or borrow qubits, nor may they call operations. It is possible, however, to pass them operations or qubits for processing. Functions are thus entirely deterministic in the sense that calling them with the same arguments will always produce the same result.

Together, operations and functions are called callables.

IQ#, the Q# Jupyter kernel, allows you to write Q# functions and operations directly on a code cell and compile them by running the cell (pressing Ctrl/⌘+Enter). For example:


In [2]:
function Hello(name: String) : Unit {

    // The following line will simply write a message to the console:
    Message($"Hello {name}!");

}

Hello

When you **Run** the cell, Q# compiles the code and returns the name of the callables it found. 
In this case, it finds only one callable (`Hello`) that accepts one argument (`name` of type `String`) 
and returns no value (`Unit`, roughly equivalent to `void` in C).

As you can see, Q# is structurally very similar to familiar
languages such as C# and Java in its use of semicolons to end
statements, curly brackets to group statements, function calls and doubleslash to introduce comments. 
It is a strongly typed language so all variables, arguments and return values
must have an associated type. Q# supports a familar set of classical types like:
- Int
- BigInt
- Bool
- Double

but also some quantum-specific:
- Qubit
- Result
- Pauli

The complete set of primitive types can be found in [Q# type model documentation](https://docs.microsoft.com/en-us/quantum/language/type-model?view=qsharp-preview#primitive-types).

In classical programs a bit, or binary digit, represents the basic unit of information and can be only one of two states: `0` or `1`.

Similarly, the **qubit**, or quantum bit, represents the basic unit of information for quantum algorithms. Like a bit, a qubit when measured can only be in one of two states:  $|0\rangle$ or  $|1\rangle$; however during execution, the state of the qubit represents *the probability of reading* $|0\rangle$ or $|1\rangle$. 

> For more information about qubits and their mathematical description, 
> take a look at [the qubit topic](https://docs.microsoft.com/en-us/quantum/concepts/the-qubit?view=qsharp-preview) inside 

In Q#, $|0\rangle$ and $|1\rangle$ are represented by `Result.Zero` and `Result.One` accordingly and qubits can only be allocated inside an operation with the `using` statement. When it gets allocated a qubit is always in the $|0\rangle$  state. To measure a qubit and read its value you use the [`M`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.primitive.m) intrinsic operation.

As such, the next `HelloQuantum` operation always return `Result.Zero`:

In [3]:
operation HelloQuantum() : Result {
    using(q = Qubit()) {
        return M(q);
    }
}

HelloQuantum

To verify this, you can use the `%simulate` magic to simulate the execution of the operation in a QPU:

In [45]:
%simulate HelloQuantum

Zero

A more interesting example for a Q# operation is a quantum random number generator (QNRG). A QNRG returns a different value each time it is invoked:

In [30]:
operation QNRG() : Result {
    using (q = Qubit()) {
        H(q);
        let r = M(q);
        
        Reset(q);
        return r;
    }
}

QNRG

As mentioned, one of the key characteristics of a qubit is that during execution it's state represents the probabiliy of measuring $|0\rangle$ or $|1\rangle$. In particular, the [`H`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.primitive.h) primitive operation modifies the state of a qubit from $|0\rangle$ to a state in which there is 50% probability of measuring  $|0\rangle$ and 50% probability of measuring  $|1\rangle$.

> When the state of a qubit is such that it is not a 100% $|0\rangle$ or 100% $|1\rangle$, 
> we say the qubit is in **superposition**.

`QNRG` also demonstrate some other Q# features:
- Immutable variables are created using `let`, in the previous example, the variable `r` is assigned the value returned by invoking `M` on qubit `q`.
- When de-allocated, qubits must be explicitly set back to $|0\rangle$, otherwise the simulator will report a runtime error. An easy way to achieve this is invoking [`Reset`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.primitive.reset).

To verify the behavior of `QNRG`, we can create another Q# operation that invokes it 10 times and prints the result to the console:

In [27]:
operation Main() : Unit {
    for(i in 1..10) {
        Message($"{i}: {QNRG()}");
    }
}

Main

In [31]:
%simulate Main

1: One
2: Zero
3: One
4: One
5: One
6: Zero
7: Zero
8: Zero
9: One
10: One


()

Q# operations can return more than a `Result`, let's modify `QRNG` to return an array:

In [43]:
open Microsoft.Quantum.Arrays;

operation QNRG2(n: Int) : Result[] {

    mutable r = new Result[n];
    
    using (qs = Qubit[n]) {
        ApplyToEach(H, qs);
        
        for(i in IndexRange(qs)) {
            set r w/= i <- M(qs[i]);
        }
        
        ResetAll(qs);
    }

    return r;
}

QNRG2

`QRNG2` demonstrate other Q# features:

- `mutable` is used to initialize variables that can be modified later in the code using `set`.
- `new Result[n]` is used to initialize a new `Result` array of size `n`.


If the compiler detects any errors, it will instead show the list of errors in the output. For example:

In [2]:
operation InvalidQ() : Unit {

    // The `FooBar` operation doesn't exist, so the following line
    // will generate a `No variable with that name exists.` error:
    FooBar("Hello again!");
    
    // `Message` takes only one string argument, so the following line
    // will generate a `Unexpected argument tuple.` error:
    Message(1, 2);
}

/snippet:(5,5): error QS5022: No identifier with that name exists.
/snippet:(9,12): error QS6211: Unexpected argument tuple. Expecting an argument of type String.


Q# operations can call other operations previously defined; they can also call all the operations defined in the 
[Microsoft.Quantum.Intrinsic](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic) and 
[Microsoft.Quantum.Canon](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon) namespaces (like [Message](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.message)).

For example, you can create a new operation that calls the previously compiled `HelloQ`:


In [3]:
function HelloAndres() : Unit {
    HelloQ("Andres");
}

HelloAndres

In [4]:
%simulate HelloAndres

Hello Andres!


()

They can also use all [Q# standard library](https://docs.microsoft.com/qsharp/api/) operations defined in other namespaces by importing the namespace using the `open` statement. For example, to use [`PI`](https://docs.microsoft.com/qsharp/api/prelude/microsoft.quantum.math.pi) you would need to open the ` Microsoft.Quantum.Math` namespace; optionally you can call the operation providing its fully qualified name, for example:
 

In [4]:
open Microsoft.Quantum.Math;

operation HelloPi() : Unit {
    let pi = Microsoft.Quantum.Convert.DoubleAsString(PI());
    HelloQ(pi);
}

HelloPi

You can define multiple operations in a single cell and use any valid Q# code, for example:

In [10]:
open Microsoft.Quantum.Math;

/// # Summary
/// Sets the qubit's state to |+⟩
operation SetPlus(q: Qubit) : Unit {
    Reset(q);
    H(q);
}

/// # Summary
/// Sets the qubit's state to |-⟩
operation SetMinus(q: Qubit) : Unit {
    Reset(q);
    X(q);
    H(q);
}

/// # Summary
/// Randomly prepares the qubit into |+⟩ or |-⟩
operation PrepareRandomMessage(q: Qubit) : Unit {

    let choice = RandomInt(2);

    if (choice == 0) {
        Message("Prepared |-⟩");
        SetMinus(q);
    } else {
        Message("Prepared |+⟩");
        SetPlus(q);
    }
}

PrepareRandomMessage, SetMinus, SetPlus

In [44]:
?H

Microsoft.Quantum.Intrinsic.H

and create other operations that uses them:

In [6]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation NextRandomBit() : Result {
    using (q = Qubit()) {
        SetPlus(q);
        return MResetZ(q);
    }
}

operation TestPrepareQubits() : Result {
    mutable r = Zero;
    
    using (qubits = Qubit[5]) {
        ApplyToEach(PrepareRandomMessage, qubits);
        DumpMachine();
        
        set r = Measure([PauliX, PauliX, PauliX, PauliX, PauliX], qubits);
        
        ResetAll(qubits);
    }
    
    return r;
}

NextRandomBit, TestPrepareQubits

## Simulating Q# operations


Once a Q# operation has been successfully compiled, you can use the `%simulate` command to simulate it. For example:

In [7]:
%simulate HelloPi

Hello 3.14159265358979!


()

In [8]:
%simulate NextRandomBit

One

`%simulate` will print any console output on the notebook, and it will return the operation's return value. If the operation returns `Unit` it prints `()`, otherwise it prints the actual value.

`%simulate` only accepts operations that take no arguments. If you want to call an operation that accepts parameters, like `HelloQ`, create a wrapper operation that calls it with the corresponding values, like `HelloPi`.

As mentioned, all messages are printed on the notebook; this includes calls to [`DumpMachine`](https://docs.microsoft.com/en-us/qsharp/api/prelude/microsoft.quantum.extensions.diagnostics.dumpmachine):

In [9]:
%simulate TestPrepareQubits

Prepared |-⟩
Prepared |-⟩
Prepared |+⟩
Prepared |-⟩
Prepared |+⟩
# wave function for qubits with ids (least to most significant): 0;1;2;3;4
∣ 0❭:	 0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ]     --- [  0.00000 rad ]
∣ 1❭:	-0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ] ---     [  3.14159 rad ]
∣ 2❭:	-0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ] ---     [  3.14159 rad ]
∣ 3❭:	 0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ]     --- [  0.00000 rad ]
∣ 4❭:	 0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ]     --- [  0.00000 rad ]
∣ 5❭:	-0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ] ---     [  3.14159 rad ]
∣ 6❭:	-0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ] ---     [  3.14159 rad ]
∣ 7❭:	 0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ]     --- [  0.00000 rad ]
∣ 8❭:	-0.176777 +  0.000000 i	 == 	*                    [ 0.031250 ] ---     [  3.14159 rad ]
∣ 9❭:	 0.17677

One

## Estimating resources

The `%estimate` command lets you estimate the resources a given quantum operation will need to execute, without actually executing the operation. Similar to `%simulate` it takes the name of a no-arguments operation. However, `%estimate` does not keep track of the qubit's state and will not return the output of the operation, instead it returns the estimated values of how many resources, like Qubits and CNOT gates, the corresponding operation will use:

In [10]:
%estimate TestPrepareQubits

[CNOT, 0], [QubitClifford, 10], [R, 0], [Measure, 11], [T, 0], [Depth, 0], [Width, 5], [BorrowedWidth, 0]

To learn more about resources estimation, take a look at [The ResourcesEstimator Target Machine](https://docs.microsoft.com/en-us/quantum/machines/resources-estimator?) documentation.

## The Workspace

The notebook uses the folder it lives on disk to define a workspace. It will try to compile all the Q# files (i.e. all files with a `.qs` extension) it finds under the current folder and will make the operations it finds available to operations in the notebook. For example, the [Operations.qs](/edit/Operations.qs) file in this folder defines two operations:
* Microsoft.Quantum.Samples.IsMinus
* Microsoft.Quantum.Samples.IsPlus

To get the list of operations defined in the workspace, you can use the `%workspace` command:

In [11]:
%workspace

Microsoft.Quantum.Samples.IsMinus, Microsoft.Quantum.Samples.IsPlus

These operations can be used in this notebook, for example:

In [12]:
open Microsoft.Quantum.Samples;

operation CheckPlus() : Bool {
    mutable result = false;
    
    using (q = Qubit()) {
        SetPlus(q);
        set result = IsPlus(q);
        
        Reset(q);
    }

    return result;
}

CheckPlus

In [13]:
%simulate CheckPlus

True

To pick up any changes you make to a Q# file in the workspace, use `%workspace reload`. 

In [14]:
%workspace reload

Microsoft.Quantum.Samples.IsMinus, Microsoft.Quantum.Samples.IsPlus

## Getting Help ##

Q# supports adding documentation to operations via comments in the code. When such documentation exists, you can access it from the notebook by adding a question mark before or after the operation name on a code cell, for example:

In [15]:
Microsoft.Quantum.Intrinsic.X?

Microsoft.Quantum.Intrinsic.X

This documentation is available for any operations in the Prelude, Canon and Workspace, or even those defined locally in the notebook:

In [16]:
PrepareRandomMessage?

PrepareRandomMessage

## Other commands ##

### `%who`

`%who` returns the list of all local and workspace operations available.

In [17]:
%who

CheckPlus, HelloAndres, HelloPi, HelloQ, Microsoft.Quantum.Samples.IsMinus, Microsoft.Quantum.Samples.IsPlus, NextRandomBit, PrepareRandomMessage, SetMinus, SetPlus, TestPrepareQubits

### `%package`

`%package` allows you to load nuget packages and makes available any Q# operations defined on them. For example, to use the operations from [Q#'s Quantum Chemistry Library](https://docs.microsoft.com/en-us/quantum/libraries/chemistry/?view=qsharp-preview), you must load the [Microsoft.Quantum.Chemistry](https://www.nuget.org/packages/Microsoft.Quantum.Chemistry/) nuget package:

In [18]:
%package Microsoft.Quantum.Chemistry

Microsoft.Quantum.Standard::0.8.1907.1701, Microsoft.Quantum.Chemistry::0.8.1907.1701

`%package` returns the list of nuget packages currently loaded and their version.

### `%version`

`%version` simply returns the current versions of IQ# and of Jupyter Core (a library used by IQ#):

In [1]:
%version

Component,Version
iqsharp,0.9.1909.901
Jupyter Core,1.1.14623.0
